Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [ ]:
NAME = ""

---

# Barlow Twins* (`zadanie dla chętnych`)
W niniejszym zeszycie skupimy się na metodzie *Barlow Twins* (BT) [(Zbontar et al., 2021)](https://arxiv.org/abs/2103.03230). Należy się dokładnie zapoznać z ideą i zasadą działania Barlow Twins (wykorzystując materiały z wykładu oraz publikacje, bądź inne materiały dostępne w sieci). 


W niniejszym zeszycie należy wykonać zadania z zakresu implementacji metody `BT`. Model oraz trenowanie zaimplementowano z wykorzystaniem `pytorch_lightning`. Żeby zrozumieć dobrze implementacje, należy zapoznać się z klasą bazową `SSLBase` i ewaluacją na zadaniu docelowym, która została w tej klasie zaimplementowana.

In [ ]:
%load_ext tensorboard

In [ ]:
import copy

import torch
from lightning_fabric import seed_everything
from lightning.pytorch.loggers import TensorBoardLogger
from lightning import Trainer
from torch import nn, Tensor
from torch.nn import functional as F

from src.data import VisionDatamodule
from src.ssl_base import SSLBase
from src.networks import SmallConvnet, MLP
from src.augmentations import get_default_aug

# Zbiór danych
Do uczenia `Barlow Twins` wykorzystamy wypróbkowany podzbiór zbioru MNIST. Warto zaznaczyć, że jest to wybór podyktowany tylko i wyłącznie ograniczeniami w zasobach obliczeniowych. W rzeczywistości, aby otrzymać konkluzywne i rzetelne wyniki, należałoby skorzystać co najmniej ze zbioru `CIFAR-10` oraz znacznie większego modelu kodera, np. `ResNet`. Jednak do celów dydaktycznych skorzystamy z mniejszego zbioru oraz odpowiednio mniejszego kodera.

In [ ]:
# define parameters
DATA_DIR = "./data"
OUT_DIR = "./data/output/barlow_twins"
BATCH_SIZE = 64
NUM_WORKERS = 0
NUM_SAMPLES_PER_CLASS = 300

In [ ]:
datamodule = VisionDatamodule(
    root_dir=DATA_DIR,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    num_samples_per_class=NUM_SAMPLES_PER_CLASS,
)
datamodule.prepare_data()
datamodule.setup()

# Zadanie 4.1 Implementacja modelu Barlow Twins (1.5 pkt)
W poniższej komórce znajduje się częściowa implementacja modelu `Barlow Twins`. Uzupełnij brakujące implementacje funckji:
* `forward` (0.5 pkt) - implementuje przejście w przód modelu (augmentacje do dwóch widoków, forward widoków przez `encoder` oraz `projector`), zwraca parę embeddingów.
* `barlow_twins_loss` (1.0 pkt) - funkcja straty modelu Barlow Twins:
  $$\mathcal{L_{BT}} = \sum_i (1 - \mathcal{C}_{ii})^2 + \lambda \sum_i \sum_{j \neq i} \mathcal{C}_{ij}^2,$$
  gdzie $\mathcal{C}$ jest macierzą korelacji krzyżowej, którą definiujemy jako:
  $$ C_{ij} = \frac{\sum_b z_{b, i}^Az_{b, j}^B}{\sqrt{\sum_b(z_{b, i}^A)^2} \sqrt{\sum_b(z_{b, j}^B)^2}},$$
  gdzie $z_{b, i}^A$ $i$-ty element wektora embedding'ów $b$-tego obrazu z pierwszej augmentacji (augmentacji $A$); analogicznie dla drugiej augmentacji $B$.


Ponadto:
* Zwróć uwagę po jakich wymiarach liczone są korelacje w funkcji straty.
* Wykorzystuj metody z torch'a, unikaj pythonowych pętli.
* Przeanalizuj dokładnie pozostałe elementy implementacji.
* Zastanów się, czy wszystkie parametry modelu podlegają uczeniu?
* Uruchom uczenie i przeanalizuj wyniki.

In [ ]:
class BarlowTwinsModel(SSLBase):
    def __init__(
        self,
        learning_rate: float,
        weight_decay: float,
        lambda_: float,
        out_channels: int = 10,
    ):
        super().__init__(
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            out_channels=out_channels,
        )

        # Initialize online network
        self.encoder = SmallConvnet()
        self.projector = MLP(84, 84, 84, plain_last=True)

        self.aug_1 = get_default_aug()
        self.aug_2 = get_default_aug()
        
        self.lambda_ = lambda_

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        # TU WPISZ KOD
        raise NotImplementedError()
        return z_a, z_b

    def forward_repr(self, x: Tensor) -> Tensor:
        return self.encoder(x)

    def training_step(self, batch: Tensor, batch_idx: int) -> Tensor:
        x, _ = batch
        z_a, z_b = self.forward(x)
        loss = self.barlow_twins_loss(z_a=z_a, z_b=z_b)

        self.log("train/loss", loss, prog_bar=True)

        return loss

    def barlow_twins_loss(self, z_a: Tensor, z_b: Tensor) -> Tensor:
        # TU WPISZ KOD
        raise NotImplementedError()

In [ ]:
%tensorboard --logdir "./data/output/barlow_twins"

In [ ]:
# define hyperparameters
LAMBDA = 5e-3
LEARNING_RATE = 5e-4
WEIGHT_DECAY = 1e-4
EPOCHS = 100
ACCELERATOR = "cpu" # change to CUDA, if want to train on GPU

seed_everything(42)
model = BarlowTwinsModel(
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    lambda_=LAMBDA,
)
logger = TensorBoardLogger(save_dir=OUT_DIR, default_hp_metric=False)
trainer = Trainer(
    default_root_dir=OUT_DIR,
    max_epochs=EPOCHS,
    logger=logger,
    accelerator=ACCELERATOR,
    num_sanity_val_steps=0,
    log_every_n_steps=10,
)

trainer.fit(model, datamodule)
trainer.test(model, datamodule)